In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap
import numpy as np
from geopandas import GeoSeries
from shapely.geometry import Polygon


In [2]:
df_govSchools = pd.DataFrame()
for page in range(0, 11):
    url = "https://open-almaty.kz/ru/spravochnaia-informacia/obrazovanie?field_edu_cat_tid_i18n=178&page={page}".format(
        page=page)
    res = requests.get(url)
    newList = pd.read_html(res.text)[0]
    newDf = pd.DataFrame(newList)
    df_govSchools = df_govSchools.append(newDf, ignore_index=True)


In [4]:
df_govSchools.set_axis(['name', 'address', 'email', 'phone number',
                       'home number'], axis='columns', inplace=True)
print(df_govSchools.shape)

(209, 5)


In [5]:
df_govSchools.address = df_govSchools.address + ', Алматы'


In [7]:
df_govSchools.tail(10)


,name,address,email,phone number,home number
199,Школа-лицей №131,"микрорайон Самал-2, 22, Алматы",almaschooll131@mail.ru,"8 702 318 81 74, +7 727 262 12 92, +7 727 264 ...",+7 727 264 46 42
200,Школа-лицей №143,"ул. Нусупбекова, 26, Алматы",galiya_83@inbox.ru,+7 727 382 20 46,NaN
201,Школа-лицей №146,"проспект Гагарина, 311, Алматы",kgushl146@mail.ru,"8 702 605 66 55, +7 727 302 18 97",+7 727 302 18 29
202,Школа-лицей №163,"улица Назарбаева, 289, Алматы",School_163@inbox.ru,"8 775 772 02 23, +7 727 262 00 83, +7 727 262 ...",+7 727 262 94 95
203,Школа-лицей №169,"микрорайон Шанырак-1, улица Утемисова, 109, Ал...",kkgushl169@mail.ru,"8 701 148 21 93 , +7 727 263 78 88",+7 727 227 38 02
204,Школа-лицей №173,"микрорайон Мамыр-1, 21, Алматы",shkolalitsey_173@mail.ru,"8 701 791 13 11, +7 727 255 57 95, +7 727 2555886",+7 727 255 58 39
205,Школа-лицей №28,"улица Фурманова, 102, Алматы",alm_school28@mail.ru,"+7 727 272 54 68, +7 727 261 64 82, +7 727 261...",+7 727 272 69 46
206,Школа-лицей №33,"улица Калдаякова, 62, Алматы",school33medeu@mail.ru,"8777 237 72 43, +7 727 291 79 51",+7 727 291 42 99
207,Школа-лицей №48,"проспект Достык, 310Б, Алматы",schooll-licey48@mail.ru,"8705 313 4336, +7 727 381 49 86, +7 727 386-75-79",+7 727 262 13 53
208,Школа-лицей №71,"Стахановская улица, 23, Алматы",almatyschool71@mail.ru,"8 777-383-55-11, +7 727 223 44 68",+7 727 223 44 63


In [12]:
with requests.Session() as session:
    for address in df_govSchools["address"]:
        coo = geocoder.osm(address, session=session)
        if coo:
            df_govSchools.loc[df_govSchools["address"] == address, "Long"] = round(coo.osm["x"], 6)
            df_govSchools.loc[df_govSchools["address"] == address, "Lat"] = round(coo.osm["y"], 6)


In [13]:
df_govSchools.head(5)


,name,address,email,phone number,home number,Long,Lat
0,Cпециализированный лицей №126,"микрорайон Аксай-4, 66, Алматы",shl_126@mail.ru,"8 777 517 97 95, 8 747 628 05 52, +7 727 243 1...",+7 727 243 14 04,76.841019,43.227680
1,Гимназия №15,"улица Гоголя, 133, Алматы",almatygimnaziya15@mail.ru,"8 701 775 50 49, +7 727 279 51 10, 8 701 999 1...",+7 727 279 50 15,76.930876,43.259492
2,Гимназия №105,"проспект Абая, 54, Алматы",almaty-sh105@mail.ru,+7 727 392 00 59,NaN,76.911642,43.239332
3,Гимназия №111,"1-й микрорайон, 37Б, Алматы",gymn111@mail.ru,"+7 727 276 99 61, +7 727 277 12 57, +7 727 277...",+7 727 276 24 85,76.851086,43.228612
4,Гимназия №120,"улица Курмангазы, 76, Алматы",school120@almaty.kz,"8 702 563 97 53, +7 727 272 18 43, +7 727 272 ...",+7 727 272 24 12,76.938599,43.243686


In [10]:
df_govSchools[df_govSchools.Long.isnull()]


,name,address,email,phone number,home number,Long,Lat
27,КГУ «Центр адаптации для несовершеннолетних» Д...,NaN,canalmaty@mail.ru,251-46-31,NaN,NaN,NaN
28,КГУ приют для временного содержания несовершен...,"Райымбек даңғ., 405 Б, Алматы",priyut.vsn@mail.ru,246-44-89,NaN,NaN,NaN
32,Лицей №166,"Алматы, ул. Горняцкая, 5, Алматы",al_fml166@mail.ru,"8 777 261 79 83, +7 727 301 78 71",+7 727 247 28 32,NaN,NaN
33,Лицей №178,"мкр. Акбулак, Шуаткөл, 41, Алматы",licei_178@mail.ru,+7 727 392 54 42,NaN,NaN,NaN
37,Общеобразовательная школа №114,"Коунрадская улица, 12А, Алматы",shkola_114@mail.ru,"+7 727 268 71 17, +7 727 240 15 69",+7 727 268 70 15,NaN,NaN
...,...,...,...,...,...,...,...
194,Школа-гимназия №94,"пр. Гагарина 135а, Алматы",shkolagimnaziya94@mail.ru,+7 727 274 14 39,NaN,NaN,NaN
196,Школа-интернат №10 для детей из малообеспеченн...,"Каблуковк-сі, 93А, Алматы",internat_10@mail.ru,"376-55-16, 376-58-28, 377-59-99",NaN,NaN,NaN
197,Школа-интернат №17,"Достық к-сі, 266 А, Алматы",schoolint_17@mail.ru,387-26-32,NaN,NaN,NaN
199,Школа-лицей №131,"микрорайон Самал-2, 22, Алматы",almaschooll131@mail.ru,"8 702 318 81 74, +7 727 262 12 92, +7 727 264 ...",+7 727 264 46 42,NaN,NaN


In [11]:
df_govSchools.address[32] = "улица Горняцкая, 5"
df_govSchools.address[33] = "​улица Суатколь, 41"
df_govSchools.address[37] = "12А, улица Каунрадская"
df_govSchools.address[38] = "​улица Жар-жар, 54, Алматы"
df_govSchools.address[54] = "42, улица Левитана, Алматы"
df_govSchools.address[57] = "165, Айнабулак, Алматы"
df_govSchools.address[60] = "65А, Айнабулак, Алматы"
df_govSchools.address[62] = "Школа 142, улица Дунентаева, Жулдыз, Алматы"
df_govSchools.address[65] = "​улица Биянху, 87, Алматы"
df_govSchools.address[66] = "100а, Новая Садовая улица, Шапагат, Алматы"
df_govSchools.address[67] = "​улица Касыма Шарипова, 38, Алматы"
df_govSchools.address[69] = "48, Ауэзова (Ожет), Алматы"
df_govSchools.address[70] = "36, Кыдырбекова улица, Калкаман-2"
df_govSchools.address[71] = "8, Степная, Mamyr"
df_govSchools.address[73] = "96, улица Заводская (Карасу)"
df_govSchools.address[74] = " Школа, Черемушки 1-я (Карасу), Алматы"
df_govSchools.address[79] = "34, Бенберина улица, Киров, АДК, Алатауский район, Алматы"
df_govSchools.address[80] = "​микрорайон Кокжиек, 63"
df_govSchools.address[81] = " №179, Школа, 17/8, улица Шаяхметова"
df_govSchools.address[82] = "​улица Ак-Кайнар, 7"
df_govSchools.address[83] = "​микрорайон Аккент, 19"
df_govSchools.address[84] = "Общеобразовательная Школа №182, 1/65, Зердели микрорайон​"
df_govSchools.address[85] = "18, улица Жангир Хана, Бостандыкский район, Алматы​"
df_govSchools.address[86] = "184, улица Керуен, Алатауский район, Алматы​"
df_govSchools.address[87] = "185, Зулпыкар улица, Алатауский район, Алматы​"
df_govSchools.address[88] = "СШ №186, 15, Байсултанова (Акжар), Акжар​"
df_govSchools.address[89] = "2, улица Жандосова (Таусамалы)​"
df_govSchools.address[90] = "4, улица Жандосова (Таусамалы), Каменка,​"
df_govSchools.address[91] = "​​улица Казыбек Тауасарулы, 27"
df_govSchools.address[92] = "11, Abdulliny Street, Медеуский район​"
df_govSchools.address[93] = "49, Нуртазиной улица, Бостандыкский район, Алматы​"
df_govSchools.address[94] = "18, Асқаров көшесі, Каргалы, Бостандыкский район, Алматы​"
df_govSchools.address[95] = "​улица Кали Надырова, 50​"
df_govSchools.address[96] = "72/1, Шортанбай жырау (Альмерек)​"
df_govSchools.address[97] = "134, улица Жанкожа Батыра, Шанырак​"
df_govSchools.address[102] = "587а, Сабыра Рахимова улица, Шанырак 2,​"
df_govSchools.address[107] = "20, Рысқұлбеков көшесі, Бостандыкский район, Алматы​"
df_govSchools.address[109] = " 2А, Кокорай улица, Курылысши, Алатауский район, Алматы​"
df_govSchools.address[113] = "Школа №47, Байконурская улица, Турксиб, Медеуский район​"
df_govSchools.address[126] = "67 Gabdullin Street, ЖК Бухар Тауэр, Коктем​"
df_govSchools.address[127] = "Средняя шклола 7, улица Альпинистов, Алатау​"
df_govSchools.address[128] = "15А, микрорайон Казахфильм​"
df_govSchools.address[133] = "140, Zhukov St., Горный Гигант​"
df_govSchools.address[135] = "Школа №82, 6, улица Ахрименко, АДК ​"
df_govSchools.address[139] = "№88, Школа, 34, микрорайон Казахфильм​"
df_govSchools.address[144] = "​улица Ади Шарипова, 53​"
df_govSchools.address[149] = "18, Almaty Road, Jubilee, Медеуский район​"
df_govSchools.address[150] = "92, Енбек улица, Шугыла, Наурызбайский район​"
df_govSchools.address[153] = "Специализированный казахско-турецкий лицей-интернат, Сейхун, Altyn Besik, Аксай​"
df_govSchools.address[159] = "Школа №103, 158, Кадырбекулы улица, Жетысуский район​"
df_govSchools.address[161] = "47, улица Шаляпина, Kalkaman-3, Каменка,​"
df_govSchools.address[164] = "​микрорайон Айнабулак 3, 166а​"
df_govSchools.address[169] = "​11, улица Черепанова, Таугүл 2 микрорайон"
df_govSchools.address[173] = "48, Жалайыр улица, Улжан 1​"
df_govSchools.address[177] = "​улица Каркаралы, 15​"
df_govSchools.address[183] = "23, бульвар Бухар Жырау, ЖК Бухар Тауэр, Коктем​"
df_govSchools.address[194] = "​проспект Гагарина, 135а​"
df_govSchools.address[196] = "​улица Абиша Кекилбайулы, 93​"
df_govSchools.address[197] = "266А, Dostyk Avenue, Медеуский район,​"
df_govSchools.address[203] = "109, Утемисова улица, Шанырак 1​"



C:\Users\Сабина\AppData\Local\Temp\ipykernel_8848\420201090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_govSchools.address[32] = "улица Горняцкая, 5"
C:\Users\Сабина\AppData\Local\Temp\ipykernel_8848\420201090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_govSchools.address[33] = "​улица Суатколь, 41"
C:\Users\Сабина\AppData\Local\Temp\ipykernel_8848\420201090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g

In [14]:
df_govSchools = gpd.GeoDataFrame(
    df_govSchools, geometry=gpd.points_from_xy(df_govSchools.Long, df_govSchools.Lat))
df_govSchools.crs = {'init': 'epsg:4326'}
df_govSchools.head()


c:\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,name,address,email,phone number,home number,Long,Lat,geometry
0,Cпециализированный лицей №126,"микрорайон Аксай-4, 66, Алматы",shl_126@mail.ru,"8 777 517 97 95, 8 747 628 05 52, +7 727 243 1...",+7 727 243 14 04,76.841019,43.227680,POINT (76.84102 43.22768)
1,Гимназия №15,"улица Гоголя, 133, Алматы",almatygimnaziya15@mail.ru,"8 701 775 50 49, +7 727 279 51 10, 8 701 999 1...",+7 727 279 50 15,76.930876,43.259492,POINT (76.93088 43.25949)
2,Гимназия №105,"проспект Абая, 54, Алматы",almaty-sh105@mail.ru,+7 727 392 00 59,NaN,76.911642,43.239332,POINT (76.91164 43.23933)
3,Гимназия №111,"1-й микрорайон, 37Б, Алматы",gymn111@mail.ru,"+7 727 276 99 61, +7 727 277 12 57, +7 727 277...",+7 727 276 24 85,76.851086,43.228612,POINT (76.85109 43.22861)
4,Гимназия №120,"улица Курмангазы, 76, Алматы",school120@almaty.kz,"8 702 563 97 53, +7 727 272 18 43, +7 727 272 ...",+7 727 272 24 12,76.938599,43.243686,POINT (76.93860 43.24369)


In [15]:
df_govSchools = df_govSchools.dropna(subset=['Long'])


In [16]:
df_govSchools.shape


(202, 8)

In [17]:
df = gpd.read_file("map (1).geojson")

In [22]:
df.head(3)

,id,idno,kids,pl,people,geometry
0,1208.0,NaN,347,NaN,2066.0,"POLYGON ((76.90757 43.32488, 76.90609 43.32262..."
1,1207.0,NaN,285,NaN,1693.0,"POLYGON ((76.91206 43.32274, 76.90780 43.31955..."
2,1187.0,NaN,407,NaN,2223.0,"POLYGON ((76.91081 43.31842, 76.90949 43.31712..."


In [19]:
df.set_axis(['id', 'idno', 'kids', 'pl', 'people', 'geometry'],
            axis='columns', inplace=True)

In [ ]:
#df[df.idno.isnull() == False]

In [21]:
df.people[125] = 71
df.id[185] = 1645

C:\Users\Сабина\AppData\Local\Temp\ipykernel_8848\2483767215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.people[125] = 71
C:\Users\Сабина\AppData\Local\Temp\ipykernel_8848\2483767215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.id[185] = 1645


In [23]:
df.drop('idno', inplace=True, axis=1)
df.drop('pl', inplace=True, axis=1)

In [24]:
df.head(5)

,id,kids,people,geometry
0,1208.0,347,2066.0,"POLYGON ((76.90757 43.32488, 76.90609 43.32262..."
1,1207.0,285,1693.0,"POLYGON ((76.91206 43.32274, 76.90780 43.31955..."
2,1187.0,407,2223.0,"POLYGON ((76.91081 43.31842, 76.90949 43.31712..."
3,1184.0,462,2717.0,"POLYGON ((76.91498 43.32100, 76.91283 43.31819..."
4,1201.0,279,1760.0,"POLYGON ((76.91754 43.32352, 76.91509 43.32124..."


In [25]:
almaty = folium.Map(
    location=[43.2567, 76.9286],
    zoom_start=13
)
almaty = df.explore(column = "kids")
almaty_gov_schools = folium.FeatureGroup(name = "schools")
for idx, row in df_govSchools.iterrows():
    Marker([row['Lat'], row['Long']], popup=row['name']).add_to(almaty_gov_schools)



In [26]:
almaty.add_child(almaty_gov_schools)
almaty.add_child(folium.map.LayerControl())
almaty


In [27]:
almaty_map = folium.Map(
    location=[43.2567, 76.9286],
    zoom_start=13
)
almaty_map = df.explore(column="kids")
almaty_gov_schools1 = folium.FeatureGroup(name="schools")
almaty_gov_schools2 = folium.FeatureGroup(name="schools_radius")
for idx, row in df_govSchools.iterrows():
    Marker([row['Lat'], row['Long']], popup=row['name']).add_to(almaty_gov_schools1)
    folium.Circle([row['Lat'], row['Long']], radius=300).add_to(almaty_gov_schools2)
           


In [28]:
almaty_map.add_child(almaty_gov_schools1)
almaty_map.add_child(almaty_gov_schools2)
almaty.add_child(folium.map.LayerControl())
almaty_map


folium.CircleMarker([lat, lon],
                    radius=40
                   ).add_to(map)
